# Extração Automatizada de Informações de Laudos Anatomopatológicos de Mama

## Introdução Técnica Detalhada

Este notebook implementa um sistema completo para extrair, processar e validar informações clínicas relevantes de laudos anatomopatológicos de mama utilizando uma combinação de técnicas de processamento de linguagem natural (PLN) e modelos avançados de IA generativa. O processo foca na identificação de marcadores críticos para diagnóstico e classificação de lesões mamárias, particularmente relacionados à malignidade.

### Objetivo Principal

O objetivo principal deste notebook é automatizar a extração de informações estruturadas de laudos de anatomia patológica da mama, identificando:
- **Descritores de malignidade** (carcinoma, invasivo, metastático, etc.)
- **Grau histológico** (1, 2 ou 3)
- **Grau nuclear** (1, 2 ou 3)
- **Formação de túbulos** (1, 2 ou 3)
- **Índice mitótico** (1, 2 ou 3)
- **Tipo histológico** (carcinoma ductal invasivo, lobular, etc.)

Estas informações são fundamentais para o estadiamento do câncer, definição de prognóstico e direcionamento do tratamento adequado.

### Tecnologias Utilizadas

O notebook utiliza diversas tecnologias avançadas:

- **PySpark**: Framework para processamento distribuído de dados
  - `pyspark.sql`: Manipulação de dados em formato tabular
  - `pyspark.sql.functions`: Transformações e funções em colunas
  - `pyspark.sql.window`: Operações de janela para ordenação e numeração de linhas

- **OpenAI API**: Acesso ao modelo LLama-4-Maverick através de API Databricks
  - Configuração de prompts específicos para extração médica
  - Gerenciamento de temperatura e tokens para otimização de respostas

- **Processamento de Texto**:
  - `re`: Biblioteca de expressões regulares para extração baseada em padrões
  - Técnicas de normalização de texto (lowercase, remoção de caracteres especiais)

- **Manipulação de Dados**:
  - `pandas`: Para análise dos resultados e métricas
  - `numpy`: Operações numéricas
  - `tqdm`: Barras de progresso para monitoramento de processamento em lotes

- **Validação e Qualidade**:
  - Funções personalizadas para avaliação de resultados
  - Comparação entre extração baseada em regras e IA

### Fluxo de Trabalho/Etapas Principais

O notebook segue um fluxo de trabalho estruturado:

1. **Configuração do Ambiente**: 
   - Importação de bibliotecas
   - Inicialização da sessão Spark
   - Obtenção do token de autenticação Databricks

2. **Extração de Dados Fonte**:
   - Consulta SQL para selecionar laudos relevantes da tabela `refined.saude_preventiva.pardini_laudos`
   - Filtragem por linha de cuidado ('mama') e tipos específicos de exame (MICROH, FPAH)
   - Normalização dos textos dos laudos

3. **Definição de Funções de Processamento**:
   - Criação de prompt especializado para o modelo de IA
   - Implementação de funções para extrair informações via expressões regulares
   - Configuração de funções para processamento em lote

4. **Processamento com IA**:
   - Envio dos laudos para o modelo LLama-4-Maverick
   - Extração das informações estruturadas
   - Transformação das respostas em formato JSON

5. **Validação e Métricas**:
   - Comparação dos resultados da IA com extração baseada em regras (pseudo-gold)
   - Cálculo de taxas de acerto para cada campo extraído
   - Geração de relatórios de desempenho

6. **Persistência dos Resultados** (comentado no código atual):
   - Salvamento na tabela `refined.saude_preventiva.pardini_laudos_mamo_anatomia_patologica`
   - Registro de métricas de desempenho com MLflow

### Dados Envolvidos

#### Fontes de Dados:
- **Tabela Principal**: `refined.saude_preventiva.pardini_laudos`
  - Contém laudos médicos de diversos exames
  - Filtrada para linha de cuidado 'mama' e exames específicos (MICROH, FPAH)

#### Colunas Principais Utilizadas:
- `id_unidade`: Identificador da unidade médica
- `id_cliente`: Identificador do paciente
- `id_item`, `id_subitem`, `id_exame`: Identificadores do exame realizado
- `dth_pedido`: Data/hora do pedido do exame
- `laudo_tratado`: Texto completo do laudo médico
- `sigla_exame`: Código do tipo de exame

#### Tabela de Destino:
- `refined.saude_preventiva.pardini_laudos_mamo_anatomia_patologica`: Armazena os resultados da extração

### Resultados/Saídas Esperadas

O notebook produz os seguintes resultados:

1. **DataFrame Estruturado (`df_final`)**: 
   - Contém todas as informações extraídas dos laudos
   - Inclui tanto os dados originais quanto os campos extraídos

2. **Métricas de Desempenho (`json_metricas`)**:
   - Taxa de acerto para cada tipo de informação extraída
   - Comparação entre extração baseada em regras e IA

3. **Tabela Persistente** (quando descomentado):
   - Dados salvos em formato Delta para uso em análises posteriores
   - Acessível via SQL para integração com outros sistemas

### Pré-requisitos

Para executar este notebook são necessários:

- **Ambiente Databricks** com:
  - Cluster configurado com runtime ML
  - Acesso às tabelas Delta no formato `refined.saude_preventiva.*`
  - Token de acesso configurado para API de modelos (`DATABRICKS_TOKEN`)

- **Bibliotecas Python**:
  - OpenAI (para acesso à API)
  - tqdm (para barras de progresso)
  - Pandas, NumPy (para manipulação de dados)
  - PySpark (para processamento distribuído)
  - FeatureStore (para persistência, quando habilitado)

### Considerações Importantes/Observações

- **Limitação de Processamento**: O código atual inclui um limitador de registros (`limit(200)`) para fins de teste
- **Chamadas de API**: O processamento em lote está configurado para evitar sobrecarga da API (batch_size=10)
- **Validação**: A abordagem utiliza uma validação sem ground truth, comparando a extração da IA com uma extração baseada em regras
- **Dados Sensíveis**: Os laudos contêm informações médicas sensíveis, sendo importante garantir a conformidade com requisitos de privacidade
- **Código Comentado**: Algumas funcionalidades estão comentadas no código original (persistência, MLflow), necessitando descomentário para uso em produção

# Extração de dados - Anatomo Patologico
**Descritores de MALIGNIDADE:**
- carcinoma
- invasivo
- invasor
- sarcoma
- metástase
- metastático
- maligno
- maligna
- cdi, cli, cdis

Outros labels a serem extraídos:

- **Grau histológico:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"grau histológico"**.

- **Grau nuclear:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"grau nuclear"**.

- **Formação de túbulos:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"formação de túbulos"**.

- **Índice mitótico:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"mm2"**. Nesse caso, é melhor procurar o termo **"mm2"** ao invés de **"índice mitótico"**.

- **Labels de tipos histológicos:**
  - Carcinoma de mama ductal invasivo (CDI)/SOE
  - Carcinoma de mama ductal in situ
  - Carcinoma de mama lobular invasivo
  - Carcinoma de mama lobular
  - Carcinoma de mama papilífero
  - Carcinoma de mama metaplásico
  - Carcinoma de mama mucinoso
  - Carcinoma de mama tubular
  - Carcinoma de mama cístico adenoide
  - Carcinoma de mama medular
  - Carcinoma de mama micropapilar
  - Carcinoma de mama misto (ductal e lobular) invasivo

--------------------

## Instalação de Dependências

Esta célula contém comandos comentados para instalação das bibliotecas necessárias para execução do notebook. Os comandos de instalação utilizam o prefixo `%pip`, que é uma magic command do Jupyter para instalar pacotes diretamente no ambiente de execução atual. Todas as instalações estão com a flag `-q` (quiet) para minimizar o output da instalação.

As bibliotecas que seriam instaladas incluem:

- **openai**: Cliente Python para interagir com a API da OpenAI (usada para comunicação com modelos de IA)
- **tqdm**: Biblioteca para exibir barras de progresso durante operações longas
- **pandarallel**: Extensão do pandas para paralelizar operações
- **databricks-feature-store**: Cliente para interagir com o Feature Store do Databricks

Há também um comando comentado `%restart_python` que reiniciaria o kernel Python após a instalação para garantir que as novas bibliotecas sejam carregadas corretamente.

Para utilizar estas instalações, basta descomentar as linhas relevantes e executar a célula. No estado atual, a célula não realiza nenhuma operação por estar completamente comentada.

In [ ]:
# %pip install openai -q
# %pip install tqdm -q
# %pip install pandarallel -q
# %pip install databricks-feature-store -q
# %restart_python

## Importação de Bibliotecas e Configuração de Ambiente

Esta célula realiza a importação de todas as bibliotecas e módulos necessários para o funcionamento do notebook, além de configurar o ambiente de execução Spark.

### Bibliotecas Importadas

#### PySpark e SQL
- `SparkSession`: Ponto de entrada para funcionalidades do Spark
- `Window`: Funções para operações de janela (agregações, rankings)
- `row_number()`: Função para adicionar numeração sequencial a registros
- `functions` como `F`: Conjunto de funções para manipular dados em DataFrames

#### Processamento de Dados
- `pandas` e `numpy`: Para manipulação e análise de dados estruturados
- `json`: Para manipulação de objetos JSON
- `re`: Para operações com expressões regulares
- `tqdm`: Para exibição de barras de progresso em processamentos longos

#### APIs e Conectividade
- `openai`: Cliente para comunicação com APIs de modelos de IA
- `FeatureStoreClient`: Para interagir com o Feature Store do Databricks

#### Utilitários
- `os`, `sys`, `time`, `warnings`: Utilitários do sistema e controle de tempo
- `relativedelta`: Para cálculos com datas
- `List`, `Any` (typing): Para tipagem estática em funções Python

### Configurações Realizadas

A célula também:

1. **Inicializa a Sessão Spark**: Cria uma nova sessão SparkSession configurada com o nome "LLM_Extractor"
2. **Obtém Token Databricks**: Recupera o token de API do contexto atual do notebook para uso nas chamadas à API do LLM

O token obtido (`DATABRICKS_TOKEN`) será utilizado posteriormente para autenticação com a API de modelos de linguagem na infraestrutura Databricks.

### Impacto

Esta célula prepara todo o ambiente necessário para as operações subsequentes de extração e processamento de dados. Após sua execução, todas as bibliotecas e configurações estarão disponíveis para uso nas demais células do notebook.

In [ ]:
from pyspark.sql import SparkSession
import json
import re
import os
import sys
import time
import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
from typing import List, Any
import openai
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from databricks.feature_store import FeatureStoreClient

spark = SparkSession.builder.appName("LLM_Extractor").getOrCreate()

DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get() if dbutils.notebook.entry_point.getDbutils().notebook().getContext() is not None else None

## Configuração de Opções de Exibição do Pandas (Comentado)

Esta célula contém código comentado para configurar opções de exibição do pandas. Quando descomentado, o código ajustaria a forma como o pandas exibe DataFrames no notebook, expandindo os limites padrão de visualização.

As opções comentadas são:

- `pd.set_option('display.max_rows', None)`: Remove o limite de linhas exibidas, mostrando todas as linhas do DataFrame
- `pd.set_option('display.max_columns', None)`: Remove o limite de colunas exibidas, mostrando todas as colunas do DataFrame
- `pd.set_option('display.width', None)`: Remove a limitação de largura da exibição, permitindo que o conteúdo use todo o espaço horizontal disponível
- `pd.set_option('display.max_colwidth', None)`: Remove o limite de largura de cada coluna, exibindo o conteúdo completo de cada célula sem truncamento

Estas configurações são particularmente úteis ao trabalhar com dados textuais longos, como os laudos médicos que serão processados neste notebook, permitindo a visualização completa do conteúdo durante análises exploratórias.

Para ativar estas configurações, basta descomentar as linhas relevantes antes de executar a célula.

In [ ]:
# pd.set_option('display.max_rows', None)       
# pd.set_option('display.max_columns', None)   
# pd.set_option('display.width', None) 
# pd.set_option('display.max_colwidth', None)

## Extração e Transformação de Dados de Laudos Anatomopatológicos

Esta célula realiza a extração, filtragem e transformação dos dados de laudos anatomopatológicos da base do Pardini, preparando o dataset que será processado pelo modelo de IA.

### Objetivo da Célula

Esta célula:
1. Define duas consultas SQL para extração de dados
2. Executa a consulta principal para obter laudos anatomopatológicos
3. Aplica transformações adicionais como filtragem por topografia e normalização de texto
4. Adiciona um índice sequencial para rastreamento dos registros

### Consultas SQL Definidas

#### `query_append` (Não utilizada nesta execução)
Esta consulta foi definida para extrair apenas registros incrementais, utilizando:
- Join com a tabela de destino para identificar novos registros
- Filtragem por linha de cuidado "mama" e códigos específicos de exame
- Está comentada/não utilizada na execução atual

#### `query_all_base` (Utilizada nesta execução)
Esta consulta seleciona todos os registros relevantes da tabela `refined.saude_preventiva.pardini_laudos`:
- Filtra por linha de cuidado = 'mama'
- Seleciona apenas exames com siglas específicas: "MICROH" e "FPAH"
- Extrai colunas de identificação e o texto do laudo

### Transformações Aplicadas

Após a execução da consulta, o DataFrame resultante passa por três transformações principais:

1. **Filtragem por Topografia**:
   ```python
   df_anatomopatologico = df_anatomopatologico.filter(F.col("laudo_tratado").rlike("(?i)Topografia: mama"))
   ```
   Este filtro utiliza uma expressão regular case-insensitive para garantir que apenas laudos relacionados à mama sejam processados.

2. **Normalização do Texto**:
   ```python
   df_anatomopatologico = df_anatomopatologico.withColumn("laudo_tratado", F.lower(df_anatomopatologico["laudo_tratado"]))
   ```
   Converte todo o texto do laudo para minúsculas, facilitando a posterior extração por expressões regulares.

3. **Adição de Índice Sequencial**:
   ```python
   window = Window.orderBy(F.monotonically_increasing_id())
   df_anatomopatologico = df_anatomopatologico.withColumn("index", row_number().over(window) - 1)
   ```
   Adiciona uma coluna "index" com numeração sequencial iniciando em 0, que será utilizada para junção com os resultados da extração por IA.

### Resultado

O resultado desta célula é o DataFrame `df_anatomopatologico` contendo os laudos filtrados e normalizados, com uma coluna adicional de índice. Este DataFrame será a base para o processamento de extração de informações através do modelo de IA.

In [ ]:
query_append = """
WITH base AS (
    SELECT
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_item, 
        flr.id_subitem, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.laudo_tratado,
        flr.sigla_exame,
        flr.linha_cuidado
        FROM refined.saude_preventiva.pardini_laudos flr
    WHERE
        flr.linha_cuidado   = 'mama'
        flr.sigla_exame IN ("MICROH","MICROH","FPAH")
),
sem_extracao AS (
    SELECT
        b.id_unidade,
        b.id_cliente,
        b.id_item,
        b.id_subitem,
        b.id_exame,
        b.dth_pedido,
        b.sigla_exame,
        b.laudo_tratado,
        b.RAW_CARCINOMA,
        b.HAS_CARCINOMA
    FROM base b
    LEFT JOIN refined.saude_preventiva.pardini_laudos_mamo_anatomia_patologica mb
      ON mb.id_unidade = b.id_unidade
     AND mb.id_cliente = b.id_cliente
     AND mb.id_item    = b.id_item
     AND mb.id_subitem = b.id_subitem
     AND mb.id_exame   = b.id_exame
    WHERE mb.id_unidade IS NULL
)
SELECT *
FROM sem_extracao
"""

query_all_base = """SELECT
        flr.id_unidade,
        -- flr.id_pedido,
        flr.id_cliente, 
        id_item, 
        id_subitem, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.laudo_tratado,
        flr.sigla_exame
    FROM refined.saude_preventiva.pardini_laudos flr
    WHERE
      flr.linha_cuidado = 'mama'
      AND
      flr.sigla_exame IN ("MICROH","MICROH","FPAH")
      """
df_anatomopatologico = spark.sql(query_all_base)
df_anatomopatologico = df_anatomopatologico.filter(F.col("laudo_tratado").rlike("(?i)Topografia: mama"))
df_anatomopatologico = df_anatomopatologico.withColumn("laudo_tratado", F.lower(df_anatomopatologico["laudo_tratado"]))
window = Window.orderBy(F.monotonically_increasing_id())
df_anatomopatologico = df_anatomopatologico.withColumn("index", row_number().over(window) - 1)

## Contagem de Registros para Processamento

Esta célula realiza uma operação simples mas importante: conta o número de registros no DataFrame `df_anatomopatologico` após a filtragem e transformação realizadas anteriormente. Esta contagem é armazenada na variável `num_linhas`.

### Objetivo da Célula

O principal objetivo é quantificar o volume de dados que será processado pelo modelo de IA, permitindo:
1. Verificar se há dados disponíveis para processamento
2. Estimar o tempo e recursos necessários para o processamento completo
3. Servir como base para decisões condicionais em células posteriores

### Funcionamento

O comando `df_anatomopatologico.count()` executa uma ação em Spark que percorre todo o DataFrame e conta o número de linhas. Esta é uma operação que materializa o DataFrame, já que as transformações anteriores são lazy (só executadas quando necessário).

### Impacto

O valor armazenado em `num_linhas` será utilizado posteriormente em uma condição `if num_linhas > 0:` para determinar se o processamento com o modelo de IA deve prosseguir. Isso evita tentativas de processamento quando não há dados disponíveis.

In [ ]:
num_linhas = df_anatomopatologico.count()

## Definição de Funções para Processamento de Laudos com IA

Esta célula implementa um conjunto completo de funções para processar laudos médicos utilizando modelos de IA. É uma das células mais complexas do notebook, incluindo definições de prompt, lógica de comunicação com API e tratamento de erros.

### Funções Implementadas

#### 1. `prompt_laudo(laudo_texto: str) -> str`

**Objetivo**: Criar um prompt estruturado para extração de informações específicas de um laudo médico.

**Parâmetros**:
- `laudo_texto`: O texto do laudo a ser analisado

**Funcionamento**:
- Constrói um prompt detalhado com instruções específicas para o modelo de IA
- Inclui o texto do laudo dentro de aspas triplas
- Define critérios claros para extração de cada tipo de informação (descritores de malignidade, grau histológico, etc.)
- Especifica o formato esperado para a resposta (dicionário Python)

**Saída**: String contendo o prompt completo a ser enviado para o modelo de IA

#### 2. `generate(descricao_agente:str, laudo:str, llm_client) -> str`

**Objetivo**: Processar um único laudo através do modelo de IA.

**Parâmetros**:
- `descricao_agente`: Descrição do papel que o modelo deve assumir (médico oncologista)
- `laudo`: Texto do laudo a ser analisado
- `llm_client`: Cliente da API do modelo de IA

**Funcionamento**:
- Gera o prompt chamando a função `prompt_laudo()`
- Configura os parâmetros da chamada de API:
  - Modelo: "databricks-llama-4-maverick"
  - Temperature: 0 (para maximizar a determinação das respostas)
  - Tokens máximos: 4000
  - Configurações adicionais como top_p, frequency/presence penalty
- Implementa lógica de retry (até 3 tentativas) em caso de falha de conexão
- Captura e trata exceções, com reporte de erros

**Saída**: Resposta textual do modelo de IA contendo o resultado da extração

#### 3. `batch_generate(descricao_agente, laudos, llm_client, batch_size=25)`

**Objetivo**: Processar múltiplos laudos em lotes, controlando a carga sobre a API.

**Parâmetros**:
- `descricao_agente`: Descrição do papel do modelo
- `laudos`: Lista de textos de laudos a serem analisados
- `llm_client`: Cliente da API
- `batch_size`: Tamanho do lote para processamento (padrão: 25)

**Funcionamento**:
- Reconfigura o cliente OpenAI com o token e URL do endpoint Databricks
- Divide o processamento em lotes do tamanho especificado
- Utiliza a biblioteca tqdm para exibir barras de progresso do processamento
- Chama a função `generate()` para cada laudo individualmente

**Saída**: Lista de respostas do modelo, uma para cada laudo processado

#### 4. `limpar_e_converter(item)`

**Objetivo**: Converter a resposta textual do modelo em um dicionário Python estruturado.

**Parâmetros**:
- `item`: Texto de resposta do modelo de IA

**Funcionamento**:
- Remove marcadores Markdown para blocos de código (```python, ```)
- Converte a string JSON para um objeto Python utilizando json.loads()
- Implementa tratamento de erros, retornando um dicionário padrão em caso de falha
- O dicionário padrão contém todos os campos com valores vazios ou "NÃO INFORMADO"

**Saída**: Dicionário Python contendo as informações extraídas estruturadas

### Impacto

Este conjunto de funções forma o núcleo do processamento de laudos com IA, permitindo:
1. Extrair informações estruturadas de texto livre
2. Processar grandes volumes de laudos de forma eficiente
3. Padronizar as respostas em formato consistente
4. Tratar erros e falhas de forma robusta

O design modular facilita a manutenção e eventuais modificações nos critérios de extração ou parâmetros do modelo.

In [ ]:
def prompt_laudo(laudo_texto: str) -> str:
    prompt = f"""A seguir está um laudo médico de mamografia. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Sempre retorne apenas o dicionário Python.

    Laudo clínico:
    \"\"\"{laudo_texto}\"\"\"

    ### Critérios de extração:

    - **Descritores de malignidade**: retorne uma **lista** com os termos de malignidade encontrados no texto (case-insensitive). Se nenhum for encontrado, retorne lista vazia `[]`. Lista de termos: ['carcinoma', "invasivo", "invasor", "sarcoma", "metástase", "metastático", "maligno", "maligna", "cdi", "cli", "cdis"]

    - **Grau histológico**: retorne o valor numérico do grau histológico.

    - **Grau nuclear**: retorne o valor numérico do grau nuclear.

    - **Formação de túbulos**: retorne o valor numérico caso exista formação de túbulos.

    - **Índice mitótico**: retorne o valor numérico do score do índice mitótico que aparece após o mm2.

    - **Tipo histológico**: identifique e retorne a frase correspondente se algum dos seguintes for mencionado (case-insensitive, variações aceitas):
      - Carcinoma de mama ductal invasivo
      - Carcinoma de mama ductal in situ
      - Carcinoma de mama lobular invasivo
      - Carcinoma de mama lobular
      - Carcinoma de mama papilífero
      - Carcinoma de mama metaplásico
      - Carcinoma de mama mucinoso
      - Carcinoma de mama tubular
      - Carcinoma de mama cístico adenoide
      - Carcinoma de mama medular
      - Carcinoma de mama micropapilar
      - Carcinoma de mama misto (ductal e lobular) invasivo

    ### Saída esperada (dicionário Python válido):
    ```python
    {{
      "descritores_malignidade": ["termo1", "termo2", ...],
      "grau_histologico": número | "NÃO INFORMADO",
      "grau_nuclear": número | "NÃO INFORMADO",
      "formacao_tubulos": número | "NÃO INFORMADO",
      "indice_mitotico": número | "NÃO INFORMADO",
      "tipo_histologico": "texto correspondente ou 'NÃO INFORMADO'
    }}
    ```
    """
    return prompt.strip()

def generate(descricao_agente:str, laudo:str, llm_client) -> str:
    """
    Gera o resultado da análise de um laudo
    Params:
        descricao_agente: descricao do agente que a LLM representa (primeira mensagem enviada à LLM)
        prompt: prompt base que será utilizado para gerar a análise
        laudo: laudo a ser analisado (incluido dentro do prompt)
        llm_client: cliente da API da LLM
    Return:
        response_message: resposta da LLM
    """
    prompt = prompt_laudo(laudo)
    messages = [
        # Utilizamos o primeiro prompt para contextualizar o llm do que ele deve fazer. 
        # No exemplo utilizamos a abordagem Role, Task, Output, Prompting.
        # Mas sintam-se a vontade para alterar de acordo com a necessidade
        {
            "role": "system",
            "content": descricao_agente
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    model_params = {
        "model": "databricks-llama-4-maverick",
        "messages": messages,
        "temperature": 0,
        "max_tokens": 4000,
        "top_p": 0.75,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
    connection_retry = 0
    while connection_retry < 3:
        try:
            response = llm_client.chat.completions.create(**model_params)
            response_message = response.choices[0].message.content
            break
        # TODO: verificar se a excessao é de conexao
        except (ConnectionError, TimeoutError) as e:
            connection_retry += 1
            print("Sem reposta do modelo")
            print(str(e))
            print("Tentando novamente...")
            time.sleep(0.1)
        except Exception as e:
            raise e

    if connection_retry >= 3:
        response_message = ''
    
    return response_message


def batch_generate(descricao_agente, laudos, llm_client, batch_size=25):
    responses = []
    
    llm_client = openai.OpenAI(
        api_key=DATABRICKS_TOKEN,
        base_url="https://dbc-d80f50a9-af23.cloud.databricks.com/serving-endpoints"
    )
    
    # Dividir em lotes
    for i in range(0, len(laudos), batch_size):
        laudos_batch = laudos[i:i+batch_size]
        for laudo in tqdm(laudos_batch, desc=f"Processando lote {i//batch_size + 1}", total=len(laudos_batch)):
            responses.append(generate(descricao_agente, laudo, llm_client))
    
    return responses

def limpar_e_converter(item):
    try:
        item_limpo = re.sub(r"```(?:python)?", "", item).replace("```", "").strip()
        return json.loads(item_limpo)
    except Exception as e:
        print(f"Erro ao converter resposta: {e}")
        return {
            'descritores_malignidade': [],
            'grau_histologico': "NÃO INFORMADO",
            'grau_nuclear': "NÃO INFORMADO",
            'formacao_tubulos': "NÃO INFORMADO",
            'indice_mitotico': "NÃO INFORMADO",
            'tipo_histologico': "NÃO INFORMADO"
        }

## Funções para Extração Baseada em Regras e Validação de Resultados

Esta célula implementa um conjunto abrangente de funções para extrair informações dos laudos utilizando expressões regulares (abordagem baseada em regras) e para validar as respostas do modelo de IA contra essa extração.

### Funções de Extração por Expressões Regulares

#### 1. Lista de Termos e Tipos
Duas listas são definidas:
- `TERMS`: Lista de descritores de malignidade a serem identificados nos laudos
- `TIPOS`: Lista de tipos histológicos a serem identificados nos laudos

#### 2. Funções Específicas para Cada Campo

##### `extrai_descritores(txt)`
- **Objetivo**: Identificar todos os descritores de malignidade presentes no texto
- **Método**: Para cada termo na lista `TERMS`, verifica sua presença no texto usando expressão regular com fronteira de palavra (`\\b`)
- **Saída**: Lista ordenada de termos encontrados

##### `extrai_grau_histologico(txt)`
- **Objetivo**: Extrair o grau histológico (1, 2 ou 3) do texto
- **Método**: Usa expressão regular para identificar padrões como "grau histológico: 2" ou variações
- **Saída**: Valor numérico ou `None` se não encontrado

##### `extrai_grau_nuclear(txt)`
- **Objetivo**: Extrair o grau nuclear (1, 2 ou 3) do texto
- **Método**: Similar ao grau histológico, procura por padrões específicos
- **Saída**: Valor numérico ou `None` se não encontrado

##### `extrai_formacao_tubulos(txt)`
- **Objetivo**: Extrair a classificação de formação de túbulos (1, 2 ou 3)
- **Método**: Busca padrões como "formação de túbulos: 2" com tratamento para variações de escrita
- **Saída**: Valor numérico ou `None` se não encontrado

##### `extrai_indice_mitotico(txt)`
- **Objetivo**: Extrair o índice mitótico do texto
- **Método**: Busca por padrões como "mitótico: 2 mm2" ou "índice mitótico 3/10 mm2"
- **Saída**: Valor numérico ou `None` se não encontrado

##### `extrai_tipo_histologico(txt)`
- **Objetivo**: Identificar o tipo histológico mencionado no texto
- **Método**: Converte o texto para minúsculas e verifica a presença de cada tipo da lista `TIPOS`
- **Saída**: String com o tipo histológico ou `None` se não encontrado

### Função de Validação

#### `avalia_extracao_sem_ground_truth(laudo_texto, json_modelo)`
- **Objetivo**: Comparar os resultados da extração por IA com uma extração baseada em regras
- **Método**: 
  1. Gera um "pseudo-gold standard" usando as funções de extração por regras
  2. Compara cada campo extraído pela IA com o correspondente do pseudo-gold
  3. Para descritores de malignidade, compara os conjuntos de termos
  4. Para campos numéricos/texto, compara os valores exatos
- **Saída**: 
  1. O dicionário completo do pseudo-gold
  2. Um dicionário de comparações contendo, para cada campo:
     - O valor do pseudo-gold
     - O valor extraído pela IA
     - Um booleano indicando se houve acerto

### Impacto e Uso

Este conjunto de funções é fundamental para:

1. **Validação Automática**: Permite avaliar o desempenho do modelo de IA sem necessidade de anotação manual
2. **Métricas de Qualidade**: Fornece base para calcular taxas de acerto por campo
3. **Diagnóstico de Erros**: Possibilita identificar padrões específicos onde a IA apresenta dificuldades

Esta abordagem de "validação sem ground truth" é especialmente valiosa em cenários onde a criação de um conjunto de dados de referência manualmente anotado seria custosa e demorada.

In [ ]:
TERMS = ['carcinoma', 'invasivo', 'invasor', 'sarcoma', 
         'metástase', 'metastático', 'maligno', 'maligna', 
         'cdi', 'cli', 'cdis']

def extrai_descritores(txt):
    achados = set()
    for termo in TERMS:
        # insensível a maiúsculas e minúsculas, plenos caracteres
        if re.search(rf"\b{re.escape(termo)}\b", txt, flags=re.IGNORECASE):
            achados.add(termo.lower())
    return sorted(achados)  # lista em ordem alfabética

def extrai_grau_histologico(txt):
    # Captura algo como "Grau histológico: 2" ou "grau histológico 2"
    m = re.search(r"grau\s+histol[oó]gico\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
    if m:
        return int(m.group(1))
    return None

def extrai_grau_nuclear(txt):
    m = re.search(r"grau\s+nuclear\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
    return int(m.group(1)) if m else None

def extrai_formacao_tubulos(txt):
    m = re.search(r"forma[cç][aã]o\s+de\s+t[uú]bulos\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
    return int(m.group(1)) if m else None

def extrai_indice_mitotico(txt):
    # Ex.: "índice mitótico 3/10 mm2" ou "mitótico: 2 mm2"
    m = re.search(r"mit[oó]tico\s*[:\-]?\s*(\d+)\s*/?\s*\d*\s*mm2", txt, flags=re.IGNORECASE)
    if m:
        return int(m.group(1))
    return None

TIPOS = [
  "carcinoma de mama ductal invasivo",
  "carcinoma de mama ductal in situ",
  "carcinoma de mama lobular invasivo",
  "carcinoma de mama lobular",
  "carcinoma de mama papilífero",
  "carcinoma de mama metapl[aá]sico",
  "carcinoma de mama mucinoso",
  "carcinoma de mama tubular",
  "carcinoma de mama c[ií]stico adenoide",
  "carcinoma de mama medular",
  "carcinoma de mama micropapilar",
  "carcinoma de mama misto (ductal e lobular) invasivo"
]

def extrai_tipo_histologico(txt):
    txt_lower = txt.lower()
    for tipo in TIPOS:
        # usar comparação simplificada, removendo acentos se quiser
        padrao = tipo.lower()
        if padrao in txt_lower:
            return tipo  # retorna exatamente a frase padronizada
    return None

def avalia_extracao_sem_ground_truth(laudo_texto, json_modelo):
    # 1. Gera pseudo-gold
    descrs_hei = extrai_descritores(laudo_texto)
    gr_hist_hei = extrai_grau_histologico(laudo_texto)
    gr_nuc_hei  = extrai_grau_nuclear(laudo_texto)
    form_tub_hei= extrai_formacao_tubulos(laudo_texto)
    ind_mit_hei = extrai_indice_mitotico(laudo_texto)
    tipo_histo_hei = extrai_tipo_histologico(laudo_texto)

    json_heu = {
        "descritores_malignidade": descrs_hei,
        "grau_histologico": gr_hist_hei if gr_hist_hei is not None else "NÃO INFORMADO",
        "grau_nuclear": gr_nuc_hei if gr_nuc_hei is not None else "NÃO INFORMADO",
        "formacao_tubulos": form_tub_hei if form_tub_hei is not None else "NÃO INFORMADO",
        "indice_mitotico": ind_mit_hei if ind_mit_hei is not None else "NÃO INFORMADO",
        "tipo_histologico": tipo_histo_hei if tipo_histo_hei is not None else "NÃO INFORMADO"
    }

    # 2. Prepara json_modelo – já é recebido do ChatGPT como dicionário Python

    # 3. Comparações campo a campo:
    comparacoes = {}

    # 3.1. Descritores de malignidade: compara igualdade exata (acertos ou não)
    val_heu_desc = set(json_heu["descritores_malignidade"])
    val_mod_desc = set(json_modelo.get("descritores_malignidade", []))
    acertou_desc = (val_heu_desc == val_mod_desc)
    comparacoes["descritores_malignidade"] = {
        "pseudo_gold": json_heu["descritores_malignidade"],
        "IA": json_modelo.get("descritores_malignidade", []),
        "acertou": acertou_desc
    }

    # 3.2. Para cada campo numérico ou de texto, basta verificar igualdade exata
    def compara_campo(nome):
        val_heu = json_heu[nome]
        val_mod = json_modelo.get(nome, "NÃO INFORMADO")
        acertou = (val_heu == val_mod)
        return {
            "pseudo_gold": val_heu,
            "IA": val_mod,
            "acertou": acertou
        }

    for campo in ["grau_histologico", "grau_nuclear", "formacao_tubulos", "indice_mitotico", "tipo_histologico"]:
        comparacoes[campo] = compara_campo(campo)

    return json_heu, comparacoes

## Função para Agregação e Análise de Resultados de Validação

Esta célula implementa a função `agrega_resultados()` que processa os resultados das comparações individuais entre as extrações do modelo de IA e o pseudo-gold standard, gerando métricas consolidadas de desempenho.

### Objetivo da Função

A função `agrega_resultados()` tem como objetivo calcular estatísticas agregadas sobre o desempenho do modelo de IA na extração de informações dos laudos, incluindo:
- Contagem total de acertos por campo
- Taxa de acerto (percentual de sucesso) para cada campo
- Métricas gerais sobre o volume de dados processados

### Parâmetros e Estrutura

**Parâmetro de Entrada**:
- `lista_comparacoes`: Lista de dicionários, cada um contendo os resultados da comparação para um laudo individual

**Fluxo de Processamento**:
1. Calcula o total de laudos processados (`total_laudos = len(lista_comparacoes)`)
2. Inicializa contadores para rastrear acertos:
   - `acertos_descritores` para descritores de malignidade
   - `acertos_campos` usando `Counter()` para demais campos
3. Itera pela lista de comparações:
   - Verifica e incrementa contador se houve acerto em descritores de malignidade
   - Para cada campo (grau histológico, nuclear, etc.), verifica e conta acertos
4. Constrói um dicionário de resultados com estatísticas para cada campo

**Estrutura do Resultado**:
Para cada campo (descritores_malignidade, grau_histológico, etc.), a função gera:
```python
{
    "acertos": número_de_acertos,
    "total": total_de_laudos,
    "taxa_acerto": número_de_acertos / total_de_laudos
}
```

### Campos Analisados

A função calcula métricas para todos os campos relevantes:
- Descritores de malignidade
- Grau histológico
- Grau nuclear
- Formação de túbulos
- Índice mitótico
- Tipo histológico

### Importância e Uso

Este tipo de função de agregação é essencial para:
1. Avaliar o desempenho global do modelo de IA na tarefa de extração
2. Identificar campos específicos onde o modelo pode estar enfrentando dificuldades
3. Acompanhar a evolução do desempenho ao longo do tempo ou com diferentes configurações de prompt

As métricas geradas podem ser usadas para:
- Reportar o desempenho do modelo para stakeholders
- Definir thresholds de confiança para uso em produção
- Identificar oportunidades de melhoria no prompt ou no processamento

In [ ]:
from collections import Counter

def agrega_resultados(lista_comparacoes):
    total_laudos = len(lista_comparacoes)
    
    # Conta quantos acertos em descritores_malignidade
    acertos_descritores = 0
    
    # Conta acertos por campo numérico/textual
    acertos_campos = Counter()
    
    for comp in lista_comparacoes:
        # Para descritores_malignidade, só existe "acertou"
        if comp["descritores_malignidade"]["acertou"]:
            acertos_descritores += 1
        
        # Para cada campo numérico/textual
        for campo in [
            "grau_histologico", 
            "grau_nuclear", 
            "formacao_tubulos", 
            "indice_mitotico", 
            "tipo_histologico"
        ]:
            if comp[campo]["acertou"]:
                acertos_campos[campo] += 1

    resultado = {
        "descritores_malignidade": {
            "acertos": acertos_descritores,
            "total": total_laudos,
            "taxa_acerto": acertos_descritores / total_laudos if total_laudos > 0 else 0.0
        }
    }

    for campo in [
        "grau_histologico", 
        "grau_nuclear", 
        "formacao_tubulos", 
        "indice_mitotico", 
        "tipo_histologico"
    ]:
        acertou = acertos_campos[campo]
        resultado[campo] = {
            "acertos": acertou,
            "total": total_laudos,
            "taxa_acerto": acertou / total_laudos if total_laudos > 0 else 0.0
        }

    return resultado

## Processamento Principal e Geração de Métricas

Esta célula contém o núcleo do processamento do notebook, onde todas as funções definidas anteriormente são utilizadas para extrair, processar, validar e analisar os dados dos laudos anatomopatológicos. Este bloco de código é executado condicionalmente, apenas se houver dados disponíveis para processamento (`num_linhas > 0`).

### Fluxo de Processamento

O código segue um fluxo sequencial lógico:

1. **Inicialização do Cliente de API**:
   - Configura o cliente OpenAI usando o token Databricks e o endpoint específico
   - Define o papel do modelo como "médico oncologista especialista em laudos de mamografia"

2. **Limitação do Dataset** (para testes):
   ```python
   df_anatomopatologico = df_anatomopatologico.limit(200)
   ```
   Esta linha limita o processamento a 200 laudos para fins de teste. Em um ambiente de produção, esta linha seria removida ou ajustada.

3. **Extração dos Laudos**:
   - Coleta os textos dos laudos do DataFrame em uma lista Python
   - Facilita o processamento em lote pelo modelo de IA

4. **Processamento com o Modelo de IA**:
   - Chama a função `batch_generate()` com tamanho de lote 10
   - Processa todos os laudos coletados, gerando respostas estruturadas

5. **Conversão e Limpeza das Respostas**:
   - Aplica a função `limpar_e_converter()` a cada resposta
   - Transforma as respostas textuais em dicionários Python estruturados

6. **Criação de DataFrame com Resultados**:
   - Define um schema Spark para os dados extraídos
   - Cria um novo DataFrame com as extrações do modelo

7. **Indexação e Join dos DataFrames**:
   - Adiciona um índice sequencial a ambos os DataFrames
   - Realiza um join para combinar os dados originais com as extrações

8. **Avaliação e Validação**:
   - Compara cada extração com seu correspondente "pseudo-gold"
   - Gera métricas detalhadas para cada campo extraído

9. **Análise de Resultados**:
   - Cria um DataFrame de métricas com informações detalhadas
   - Gera um resumo agregado de desempenho com `agrega_resultados()`

### Aspectos Importantes

#### Código Comentado

A célula contém várias seções comentadas que representam funcionalidades potencialmente úteis:

1. **Criação da Tabela Feature Store**:
   ```python
   #fs = FeatureStoreClient()
   #fs.create_table(...)
   ```
   Este código criaria uma tabela permanente para armazenar os resultados.

2. **Persistência dos Resultados**:
   ```python
   #fs = FeatureStoreClient()
   #if num_linhas > 0:
   #    print(f"Há {num_linhas} registros para inserir — executando gravação…")
   #    ...
   ```
   Este código salvaria os resultados na tabela Feature Store.

3. **Registro de Métricas no MLflow**:
   ```python
   # mlflow.set_experiment(...)
   # with mlflow.start_run(run_name="Extracao_Laudos_Run_Threshold"):
   #     ...
   ```
   Este código registraria as métricas de desempenho no MLflow para acompanhamento.

#### Resultados Gerados

A execução desta célula produz:

1. **DataFrame Final (`df_final`)**:
   - Contém os dados originais dos laudos
   - Inclui todas as extrações do modelo de IA

2. **Métricas Detalhadas (`df_metrics`)**:
   - DataFrame pandas com resultados detalhados para cada laudo
   - Inclui comparações entre extração baseada em regras e IA

3. **Métricas Agregadas (`json_metricas`)**:
   - Resumo do desempenho da extração
   - Taxas de acerto por campo extraído

### Considerações de Desempenho

- O processamento está limitado a 200 registros para testes
- O tamanho de lote está configurado para 10, o que equilibra eficiência e carga na API
- As etapas de validação são computacionalmente intensivas mas necessárias para garantir qualidade

In [ ]:
if num_linhas > 0:
    llm_client = openai.OpenAI(api_key=DATABRICKS_TOKEN,
                           base_url="https://dbc-d80f50a9-af23.cloud.databricks.com/serving-endpoints"
                           )
    descricao_agente = "Atue como um médico oncologista especialista em laudos de mamografia."

    ###################### Apenas para testes #################
    df_anatomopatologico = df_anatomopatologico.limit(200)
    ###########################################################

    rows = df_anatomopatologico.select("laudo_tratado").collect()
    laudos = [row.laudo_tratado for row in rows]

    respostas = batch_generate(descricao_agente, laudos, llm_client, batch_size=10)

    lista_dicts = [limpar_e_converter(item) for item in respostas]

    schema = StructType([
        StructField("descritores_malignidade", ArrayType(StringType()), True),
        StructField("grau_histologico", StringType(), True),
        StructField("grau_nuclear", StringType(), True),
        StructField("formacao_tubulos", StringType(), True),
        StructField("indice_mitotico", StringType(), True),
        StructField("tipo_histologico", StringType(), True),
    ])

    df_lista = spark.createDataFrame(lista_dicts, schema=schema)

    w = Window.orderBy(F.lit(1))

    df_imuno_indexed = (
        df_anatomopatologico
        .withColumn("row_id", F.row_number().over(w) - 1)  # subtrai 1 para ficar zero‐based
    )

    df_lista_indexed = (
        df_lista
        .withColumn("row_id", F.row_number().over(w) - 1)
    )

    df_final = df_imuno_indexed.join(df_lista_indexed, on="row_id").drop("row_id")

    # Base histórica
    #fs = FeatureStoreClient()
    #fs = FeatureStoreClient()
    #fs.create_table(
    #    name="refined.saude_preventiva.pardini_laudos_mamo_anatomia_patologica",
    #    primary_keys=["id_unidade", "id_cliente", "id_item", "id_subitem", "id_exame"],
    #    schema=df_final.schema,
    #    description="Features extraídas de laudos de mamografia/biopsia (Anatomia Patológica). Siglas: "
    #)

    # Append em prd
    #num_linhas = df_final.count()
    #fs = FeatureStoreClient()
    #if num_linhas > 0:
    #    print(f"Há {num_linhas} registros para inserir — executando gravação…")
    #    primary_keys = ["id_unidade", "id_cliente", "id_item", "id_subitem", "id_exame"]
    #    ###### Apenas para testes ##############
    #    df_final = df_final.dropna()
    #    df_final = df_final.dropDuplicates(primary_keys)
    #    ########################################
    #    primary_keys = ["id_unidade", "id_cliente", "id_item", "id_subitem", "id_exame"]
    #    fs.write_table(
    #        name="refined.saude_preventiva.pardini_laudos_mamo_anatomia_patologica",
    #        df=df_final,
    #        mode="merge",
    #    )
    #else:
    #    print("Nenhum registro encontrado; nada a fazer.")

    lista_laudos = laudos
    resultados = []
    for laudo_txt, json_mod in zip(lista_laudos, lista_dicts):
        pseudo_gold, compar = avalia_extracao_sem_ground_truth(laudo_txt, json_mod)
        resultados.append(compar)

    df_metrics = pd.DataFrame()
    df_metrics["laudos"] = laudos
    df_metrics["resultados"] = resultados
    resultados_expandidos = pd.json_normalize(df_metrics["resultados"])
    df_metrics = pd.concat(
        [df_metrics.drop(columns=["resultados"]), resultados_expandidos],
        axis=1
    )

    json_metricas = agrega_resultados(resultados)

    # mlflow.set_experiment("/Users/aureliano.paiva@grupofleury.com.br/anatomopatologico_pardini_metricas")

    # threshold = 0.8

    # with mlflow.start_run(run_name="Extracao_Laudos_Run_Threshold"):
    #     for campo, stats in json_metricas.items():
    #         taxa = stats["taxa_acerto"]
            
    #         # Registrar a taxa de acerto
    #         mlflow.log_metric(f"{campo}_taxa_acerto", taxa)
            
    #         # Registrar flag de aprovação no threshold
    #         passou_flag = 1 if taxa >= threshold else 0
    #         mlflow.log_metric(f"{campo}_passou_threshold", passou_flag)
            
    #         # Opcional: registrar acertos e total
    #         mlflow.log_metric(f"{campo}_acertos", stats["acertos"])
    #         mlflow.log_metric(f"{campo}_total", stats["total"])
        
    #     run_id = mlflow.active_run().info.run_id
    #     print(f"Run registrada: {run_id}")

## Exportação de Dados para Excel (Comentado)

Esta célula contém código comentado para exportar os resultados processados para um arquivo Excel. Esta funcionalidade seria útil para análise detalhada fora do ambiente Databricks ou para compartilhamento dos resultados com stakeholders que preferem trabalhar com planilhas.

### Funcionalidades Comentadas

1. **Instalação do Módulo openpyxl**:
   ```python
   #%pip install openpyxl
   ```
   O módulo `openpyxl` é necessário para que o pandas consiga exportar dados para o formato Excel (.xlsx).

2. **Conversão do DataFrame Spark para Pandas**:
   ```python
   #df_pandas = df_final.toPandas()
   ```
   Esta linha converteria o DataFrame Spark (`df_final`) para um DataFrame pandas, necessário para a exportação para Excel.

3. **Exportação para Arquivo Excel**:
   ```python
   #df_pandas.to_excel("fleury_laudos_mamografia_anatomia_patologica.xlsx", index=False)
   ```
   Esta linha salvaria os dados no arquivo especificado, sem incluir o índice do DataFrame.

### Considerações de Uso

Quando descomentado, este código:

1. Consumiria memória adicional para converter o DataFrame Spark em pandas
2. Criaria um arquivo local no ambiente de execução
3. Seria ideal para conjuntos de dados menores devido às limitações de memória

Esta funcionalidade seria particularmente útil durante a fase de desenvolvimento e validação do processamento, permitindo análises detalhadas dos resultados.

In [ ]:
#%pip install openpyxl
#df_pandas = df_final.toPandas()
#df_pandas.to_excel("fleury_laudos_mamografia_anatomia_patologica.xlsx", index=False)

## Análise Específica de Registros com Grau Histológico (Comentado)

Esta célula final contém código comentado para realizar uma análise específica dos laudos que contêm informações sobre grau histológico. O código filtraria os registros relevantes, exportaria um subconjunto para Excel e mostraria os primeiros 100 registros para análise visual.

### Funcionalidades Comentadas

1. **Configuração de Exibição Expandida do Pandas**:
   ```python
   #pd.set_option('display.max_colwidth', None)
   ```
   Esta configuração permitiria visualizar o conteúdo completo das células, sem truncamento, útil para analisar textos longos como os laudos.

2. **Filtragem de Registros com Grau Histológico**:
   ```python
   #df_pandas2 = df_pandas[df_pandas["grau_histologico"] != "NÃO INFORMADO"].head(100)
   ```
   Esta linha:
   - Filtraria apenas os registros onde o grau histológico foi extraído com sucesso
   - Limitaria o resultado aos primeiros 100 registros
   - Criaria um novo DataFrame `df_pandas2` com esse subconjunto

3. **Exportação para Excel do Subconjunto**:
   ```python
   #df_pandas2.to_excel("fleury_laudos_mamografia_anatomia_patologica_grauhistologico.xlsx", index=False)
   ```
   Salvaria o subconjunto filtrado em um arquivo Excel específico.

4. **Visualização dos Dados**:
   ```python
   #df_pandas2.head(100)
   ```
   Mostraria os registros no próprio notebook para análise visual imediata.

### Utilidade da Análise

Este código seria particularmente útil para:
1. Análise focada nos casos onde o grau histológico foi identificado
2. Validação manual da qualidade da extração desse campo específico
3. Criação de exemplos para treinamento ou ajuste do sistema

Descomentando e adaptando este código, os usuários poderiam realizar análises semelhantes para outros campos de interesse, como grau nuclear ou tipo histológico.

In [ ]:
#pd.set_option('display.max_colwidth', None)
#df_pandas2 = df_pandas[df_pandas["grau_histologico"] != "NÃO INFORMADO"].head(100)
#df_pandas2.to_excel("fleury_laudos_mamografia_anatomia_patologica_grauhistologico.xlsx", index=False)
#df_pandas2.head(100)